In [40]:
import requests
import json
from time import sleep
import pandas as pd
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from Preprocessing import Preprocessor

## API connect and get result

In [41]:
url = 'http://127.0.0.1:5000'
url_roadcast = url+ '/roadcast'
url_calc = url + '/calculation'

#with open("/home/ndsviriden/data_metro_1.txt", "r") as file:
with open("/home/ndsviriden/metro_test.txt", "r") as file:
    data = json.load(file)

In [42]:
with requests.Session() as session:
    session.headers = {'Content-type': 'application/json', 'Host': ".api.mmcc.pkcup.ru", 
                       'cache-control': "no-cache", 'postman-token': '727b926f-c7b5-854e-06d7-bea9df150a12', 
                       'User-Agent':'test'}
    
    response = session.post(url_calc, params={"key":"MeAuthKeyMeSmart"}, json=data)
    sleep(3) ## What should I do with it
    job_id = json.loads(response.text)['job_id']
    response = session.get(url_roadcast, params={"job_id": job_id, "key": "MeAuthKeyMeSmart"}, timeout=5)
    print(response.url)

KeyError: 'job_id'

In [13]:
response.status_code

500

## Create input data. Transform from .csv to .json

In [7]:
df = pd.read_csv('113_raw.csv')
station_def = pd.read_csv('station_def.csv', index_col=0)
preprocessor = Preprocessor()
df = preprocessor.PivotTable(df)
df = preprocessor.FixPressureScale(df)
df = df[['station_id', 'date_time', 'data']]
df_data = df['data']
df_id = df['date_time']
df = pd.concat((df_id, df_data), axis=1)
df['date_time'] = df['date_time'].apply(lambda x: str(x) + " UTC")
df = df.set_index('date_time')
df = df.rename(columns=transfer_dict)

FileNotFoundError: File b'113_raw.csv' does not exist

In [201]:
%%time
metro_input = {}
metro_input["rwis_data"] = df.to_dict('index')

CPU times: user 1.63 s, sys: 16 ms, total: 1.65 s
Wall time: 1.66 s


In [87]:
transfer_dict = {"t_air": "t_air", "t_road": "t_road", "t_underroad": "t_underroad",
             "dampness": "humidity", "wind_velocity": "wind_speed", "wind_speedmax": "wind_gusts",
             "wind_dir": "wind_direction", "precip_code": "precipitation_type", 
             "precip_count": "precipitation_intensity", "freezing_point": "freezing_point",
             "dew_point": "t_dew_point", "salinity": "salinity", "pressure": "pressure", 
             "visibility": "visibility", "p_weather": "p_weather", "cloudiness": "cloudiness"}

In [105]:
test = df.iloc[0]
z = test['data'].to_dict()
x = test['date_time']
type(x.item())

str

In [ ]:
rwis_data": {
    "2016-12-30 07:00 UTC": {
      "wind_speed": 4.1,
      "humidity": 99.8,
      "t_dew_point": -19.0,
      "precipitation_intensity": 0.0,
      "precipitation_type": 10,
      "salinity": 0,
      "pressure": 750.0,
      "t_road": -17.6,
      "freezing_point": 0,
      "wind_direction": 28,
      "t_air": -18.9,
      "wind_gusts": 42,
      "t_underroad": -17.0

In [129]:
%%time
{str(row[1].date_time) + " UTC": row[1]['data'].to_dict() for row in df.iterrows()}

CPU times: user 43.8 s, sys: 152 ms, total: 44 s
Wall time: 44.3 s


{'    2013-12-30 13:46:01\nName: 23659, dtype: object UTC': {'cloudiness': 10.0,
  'dampness': 983.0,
  'dew_point': -119.0,
  'freezing_point': -19.0,
  'precip_code': 2.0,
  'precip_count': 6.0,
  'pressure': 7455.0,
  'salinity': 30.0,
  't_air': -116.0,
  't_road': -79.0,
  't_underroad': -84.0,
  'wind_dir': 6.0,
  'wind_speedmax': 75.0,
  'wind_velocity': 34.0},
 '    2012-12-17 03:29:26\nName: 4472, dtype: object UTC': {'cloudiness': 8.0,
  'dampness': 961.0,
  'dew_point': -248.0,
  'freezing_point': -52.0,
  'precip_code': 0.0,
  'precip_count': 0.0,
  'pressure': 7586.0,
  'salinity': 80.0,
  't_air': -244.0,
  't_road': -186.0,
  't_underroad': -196.0,
  'wind_dir': 5.0,
  'wind_speedmax': 3.0,
  'wind_velocity': 0.0},
 '    2015-01-27 02:34:51\nName: 42178, dtype: object UTC': {'cloudiness': 3.0,
  'dampness': 886.0,
  'dew_point': -230.0,
  'freezing_point': -19.0,
  'precip_code': 0.0,
  'precip_count': 0.0,
  'pressure': 7545.0,
  'salinity': 30.0,
  't_air': -216.0,
  '

In [104]:
test = df.iloc[0]
z = test['data'].to_dict()
x = test['date_time']
rwis_data_mmx = test['data'].to_dict()
rwis_data_mmcc = {transfer_dict[x]:rwis_data_mmx[x] for x in rwis_data_mmx.keys()}
{test['date_time'].item() + " UTC": rwis_data_mmcc}

{'2012-09-17 18:01:48 UTC': {'cloudiness': 7.0,
  'freezing_point': 0.0,
  'humidity': 831.0,
  'precipitation_intensity': 0.0,
  'precipitation_type': 0.0,
  'pressure': 7357.0,
  'salinity': 0.0,
  't_air': 136.0,
  't_dew_point': 108.0,
  't_road': 140.0,
  't_underroad': 140.0,
  'wind_direction': 2.0,
  'wind_gusts': 0.0,
  'wind_speed': 0.0}}

In [60]:
test = df.iloc[0]
for column in t
rwis_data["t_air"] = test["t_air"]
rwis_data["t_road"] = test["t_road"]
rwis_data["t_underroad"] = test["t_underroad"]
rwis_data["humidity"] = test["dampness"]
rwis_data["wind_speed"] = test["wind_velocity"]
rwis_data["wind_gusts"] = test["wind_speedmax"]
rwis_data["wind_direction"] = test["wind_dir"]
rwis_data["precipitation_type"] = test["precip_code"]
rwis_data["precipitation_intensity"] = test["precip_count"]
rwis_data["freezing_point"] = test["freezing_point"]
rwis_data["t_dew_point"] = test["dew_point"]
rwis_data["salinity"] = test["salinity"]
rwis_data["pressure"] = test["pressure"]
rwis_data["visibility"] = test["visibility"]
rwis_data["p_weather"] = test["p_weather"]
rwis_data["cloudiness"] = test["cloudiness"]

KeyError: 't_air'

In [ ]:
rwis_data": {
    "2016-12-30 07:00 UTC": {
      "wind_speed": 4.1,
      "humidity": 99.8,
      "t_dew_point": -19.0,
      "precipitation_intensity": 0.0,
      "precipitation_type": 10,
      "salinity": 0,
      "pressure": 750.0,
      "t_road": -17.6,
      "freezing_point": 0,
      "wind_direction": 28,
      "t_air": -18.9,
      "wind_gusts": 42,
      "t_underroad": -17.0

(0, station_id                        103
road_id                             7
km                                 80
name          км 80+000 а/д М-4 "Дон"
longitude                     55.2789
latitude                      38.4925
altitude                            0
timezone                            3
Name: 0, dtype: object)
(1, station_id                         104
road_id                             54
km                                  47
name          км 47+000 а/д М-5 "Урал"
longitude                      55.4917
latitude                       38.1572
altitude                             0
timezone                             3
Name: 1, dtype: object)
(2, station_id                         105
road_id                             54
km                                  28
name          км 28+000 а/д М-5 "Урал"
longitude                      55.6056
latitude                       37.9789
altitude                             0
timezone                             3
Name: 2, dty

Name: 406, dtype: object)
(407, station_id                               833
road_id                                    9
km                                       473
name          км 473+073 а/д М-8 "Холмогоры"
longitude                             59.291
latitude                              39.928
altitude                                 120
timezone                                   3
Name: 407, dtype: object)
(408, station_id                               834
road_id                                    9
km                                       757
name          км 757+050 а/д М-8 "Холмогоры"
longitude                            61.2876
latitude                             42.3751
altitude                                  68
timezone                                   3
Name: 408, dtype: object)
(409, station_id                               835
road_id                                    9
km                                       804
name          км 804+500 а/д М-8 "Холмогоры"
long

Name: 504, dtype: object)
(505, station_id                                          1072
road_id                                               33
km                                                    51
name          км 51+800 а/д Р-132 (1Р-132) Калуга-Рязань
longitude                                        54.3394
latitude                                         36.8851
altitude                                             254
timezone                                               3
Name: 505, dtype: object)
(506, station_id                                           1073
road_id                                                23
km                                                     95
name          км 95+500 а/д А-130 (А-101) Москва-Рославль
longitude                                         55.1417
latitude                                          36.7918
altitude                                              100
timezone                                                3
Name: 506, dtype